# Baseline notebooks:

- Preprocessing : https://www.kaggle.com/code/motono0223/js24-preprocessing-create-lags
- Training (Code only) : https://www.kaggle.com/code/motono0223/js24-train-gbdt-model-with-lags-singlemodel
  - trained model : https://www.kaggle.com/datasets/motono0223/js24-trained-gbdt-model
- Inference : **this notebook**  https://www.kaggle.com/code/motono0223/js24-inference-gbdt-with-lags-singlemodel
- EDA(1) : https://www.kaggle.com/code/motono0223/eda-jane-street-real-time-market-data-forecasting
- EDA(2) : https://www.kaggle.com/code/motono0223/eda-v2-jane-street-real-time-market-forecasting

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os, gc
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
import pickle

from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import kaggle_evaluation.jane_street_inference_server

# Configurations

In [2]:
class CONFIG:
    seed = 42
    target_col = "responder_6"
    feature_cols = ["symbol_id", "time_id"] + [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]
    
    model_paths = [
        #"/kaggle/input/js24-train-gbdt-model-with-lags-singlemodel/result.pkl",
        "/kaggle/input/js24-trained-gbdt-model/result.pkl",
    ]

# Load preprocessed data (to calculate CV)

In [3]:
valid = pl.scan_parquet(
    f"/kaggle/input/js24-preprocessing-create-lags/validation.parquet/"
).collect().to_pandas()

In [4]:
valid

,id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,label,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
0,46045114,1670,0,0,3.405333,0.310848,1.131483,0.745982,0.776556,-0.798888,0.189267,-0.661057,-0.290370,-0.228528,11,7,76,-0.721815,0.178626,-0.625948,NaN,-0.600020,NaN,-1.773468,-1.426673,0.752113,-0.128356,1.485508,0.798556,0.939105,0.531187,1.102988,0.510059,0.409313,-0.711646,-0.902749,-0.162899,NaN,NaN,0.619876,0.466504,-1.488351,0.000466,-0.099937,NaN,0.716869,NaN,NaN,0.108630,NaN,-0.957244,1.638018,-0.578405,0.482135,0.009865,NaN,0.258550,NaN,NaN,-0.701604,NaN,-1.752637,2.125332,NaN,0.074168,0.018479,1.027483,-0.525323,-0.239548,-0.619589,-1.701620,-1.398736,-0.939831,0.298140,-0.321387,-0.737133,0.032996,-0.614959,NaN,NaN,0.811054,0.708597,-0.001946,0.004254,-0.082806,-0.082336,-0.266573,-0.125054,1.400488,-1.104670,-0.119038,1.622135,-1.964823,9,0,-0.599179,-0.421537,-0.880075,0.227441,0.181244,-0.087156,0.213826,0.138748,0.242813
1,46045115,1670,0,1,2.619899,0.382185,0.458687,0.544353,0.092550,-0.911843,0.130969,-0.905152,-0.301003,-0.187945,11,7,76,-0.920349,2.437098,-0.075225,NaN,-0.504462,NaN,-1.682780,-1.309567,1.357302,-0.007474,1.107871,0.374563,1.059325,1.151365,-1.985968,-0.465673,0.825128,-0.756643,-0.735251,-0.006619,NaN,NaN,0.495858,0.553447,-1.584693,-0.058390,0.086217,NaN,0.514809,NaN,NaN,-0.652876,NaN,-1.598468,1.125729,-0.049231,0.421088,-0.185787,NaN,1.567681,NaN,NaN,-1.137877,NaN,-1.826585,1.230211,NaN,1.252662,0.437808,1.027483,-0.342516,-0.421323,-0.424025,-1.053866,-1.873317,-0.709916,1.826658,-0.019934,-0.572781,2.331671,-0.209500,NaN,NaN,0.859362,0.641228,-0.051073,-0.032598,0.195653,-0.018512,-0.006025,-0.244250,0.459814,-1.509297,-0.339291,0.753648,-1.822841,9,0,1.206133,0.458626,-0.101498,0.050520,0.018345,-0.553546,-1.088641,-0.553292,-1.762422
2,46045116,1670,0,2,2.213198,-0.163449,0.740222,0.272134,-0.092383,-1.215421,0.137453,-0.916160,-0.251347,-0.214681,81,2,59,-1.139882,-0.194705,-0.887330,NaN,-0.232800,NaN,-1.317658,-1.673521,0.740504,-0.174841,0.804582,1.037138,0.885568,-0.109851,0.672542,0.068193,-0.324793,-0.467590,-0.299015,-0.203140,NaN,NaN,1.256832,1.047751,1.832642,-0.030375,-0.326503,NaN,0.022454,NaN,NaN,-0.266239,NaN,-1.641421,1.519165,-0.629148,0.293903,0.738224,NaN,0.604186,NaN,NaN,0.436533,NaN,-1.576730,1.866086,NaN,0.305961,0.175934,1.027483,-0.289786,-0.155150,-0.353342,-2.153739,-1.940063,-0.817407,-0.151191,-0.831711,-1.257207,-0.280923,-0.596907,NaN,NaN,0.912742,1.409243,0.007518,0.017642,-0.491809,-0.248280,-0.643635,0.213433,0.246346,0.009581,0.428176,0.858422,0.649326,9,0,-0.698454,-0.217275,-0.452308,0.005591,0.004208,-0.046340,-0.468561,-0.158282,-0.714670
3,46045117,1670,0,3,1.603719,0.362074,0.242651,0.463201,0.282871,-1.410926,0.108918,-0.962025,-0.266273,-0.191150,4,3,11,-0.921211,0.420313,-0.560784,NaN,-0.211856,NaN,-1.061166,-1.233272,-0.116152,-0.155462,0.158984,0.268396,0.917793,0.37

# Load model

In [5]:
models = []
for model_path in CONFIG.model_paths:
    with open( model_path, "rb") as fp:
        result = pickle.load(fp)

    model = result["model"]
    models.append(model)

# Show model
for model in models:
    display(model)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_gpus=2, n_jobs=None,
             num_parallel_tree=None, ...)

In [6]:
result

{'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_gpus=2, n_jobs=None,
              num_parallel_tree=None, ...),
 'cv': 0.01172717896622344,
 'cv_detail': {0: -6.509265358856098e-05,
  1: 0.008501096216896431,
  2: 0.0046989696983987805,
  3: 0.008520042469108091,
  4: 0.03054094390059725,
  5: 0.006244469946754094,
  6: 0.005099102469691141,
  7: 0.00542

# CV Score

In [7]:
X_valid = valid[ CONFIG.feature_cols ]
y_valid = valid[ CONFIG.target_col ]
w_valid = valid[ "weight" ]

X_valid.shape, y_valid.shape, w_valid.shape

((1082224, 90), (1082224,), (1082224,))

In [8]:
y_pred_valid = model.predict(X_valid)
valid_score = r2_score(y_valid, y_pred_valid, sample_weight = w_valid)
valid_score

0.01172717896622344

In [9]:
del valid, X_valid, y_valid, w_valid
gc.collect()

0

#### There seems to be bug in official code, can only submit polars dataframe

In [10]:
#lagとtrainの結合の確認
lag = pl.read_parquet("/kaggle/input/jane-street-realtime-marketdata-forecasting/lags.parquet/date_id=0")
train = pl.read_parquet("/kaggle/input/jane-street-realtime-marketdata-forecasting/train.parquet/partition_id=0/part-0.parquet")
lag

date_id,time_id,symbol_id,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,0,-0.442215,-0.322407,0.143594,-0.92689,-0.782236,-0.036595,-1.305746,-0.795677,-0.143724
0,0,1,-0.651829,-1.70784,-0.893942,-1.065488,-1.871338,-0.615652,-1.162801,-1.205924,-1.245934
0,0,2,-0.656373,-0.264575,-0.892879,-1.511886,-1.03348,-0.378265,-1.57429,-1.863071,-0.027343
0,0,3,-0.188186,-0.19097,-0.70149,0.098453,-1.015506,-0.054984,0.329152,-0.965471,0.576635
0,0,4,-0.257462,-0.471325,-0.29742,0.074018,-0.324194,-0.597093,0.219856,-0.276356,-0.90479
…,…,…,…,…,…,…,…,…,…,…,…
0,0,34,-0.185392,-0.187891,-0.206658,-0.634903,-0.643175,-0.443875,-0.556474,-1.122211,-0.884185
0,0,35,-0.308923,-0.434147,-1.354941,0.30054,-0.830827,0.424937,0.518839,-0.687369,1.440577
0,0,36,-0.074661,-0.261698,-0.007051,-2.60039,-1.146709,-1.601274,-3.216254,-1.249338,-2.868875


In [11]:
lags = lag.group_by(["date_id", "symbol_id"], maintain_order=True).last() # pick up last record of previous date
train = train.join(lags, on=["date_id", "symbol_id"],  how="left")#カラムメイがラグになっているか確認したいか
train

date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,time_id_right,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,1,3.889038,null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,…,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,0,-0.651829,-1.70784,-0.893942,-1.065488,-1.871338,-0.615652,-1.162801,-1.205924,-1.245934
0,0,7,1.370613,null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,…,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.0,0.703665,0.216683,0.778639,0,-0.114118,-0.198511,-0.200027,-0.410021,-0.135167,-0.182887,-0.492168,-0.142915,-0.202081
0,0,9,2.285698,null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,…,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,0,-0.529529,0.040104,-0.33309,-0.95904,-1.318411,-0.774299,-0.716492,-1.471419,-1.107083
0,0,10,0.690606,null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,…,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,0,-0.709064,-0.137431,-0.47596,-0.506644,-0.297788,-0.530738,-0.263427,-0.169489,-0.410877
0,0,14,0.44057,null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,…,-1.36224,null,null,null,null,null,-0.948601,-0.136814,-0.447704,-1.141761,0.099631,-0.661928,null,null,3.678076,2.793581,2.61825,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.0,-3.57282,-1.089123,-5.0,0,0.464961,0.077041,0.601805,-0.178444,1.127965,-0.445524,-0.507432,0.98516

In [12]:
lags_ : pl.DataFrame | None = None
    
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    global lags_
    if lags is not None:
        lags_ = lags

    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )
    symbol_ids = test.select('symbol_id').to_numpy()[:, 0]

    if not lags is None:
        lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last() # pick up last record of previous date
        test = test.join(lags, on=["date_id", "symbol_id"],  how="left")#カラムメイがラグになっているか確認したいかも
    else:
        test = test.with_columns(
            ( pl.lit(0.0).alias(f'responder_{idx}_lag_1') for idx in range(9) )
        )
    
    preds = np.zeros((test.shape[0],))
    for i, model in enumerate(tqdm(models)):
        preds += model.predict(test[CONFIG.feature_cols].to_pandas()) / len(models)
    print(f"predict> preds.shape =", preds.shape)
    
    predictions = \
    test.select('row_id').\
    with_columns(
        pl.Series(
            name   = 'responder_6', 
            values = np.clip(preds, a_min = -5, a_max = 5),
            dtype  = pl.Float64,
        )
    )

    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # with columns 'row_id', 'responer_6'
    assert list(predictions.columns) == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very first `predict` call, which does not have the usual 10 minute response deadline.

In [13]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/test.parquet',
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/lags.parquet',
        )
    )

  0%|          | 0/1 [00:00<?, ?it/s]

predict> preds.shape = (39,)
